In [11]:

train_srcs = './data_tmp/train_srcs.pkl'
test_srcs = './data_tmp/test_srcs.pkl'

train_txt = './data_tmp/train(12809).txt'
test_txt = './data_tmp/test(2030).txt'


In [12]:
import torch
print(torch.cuda.current_device())
print(torch.cuda.device_count())

0
4


In [13]:
import pickle

In [14]:

train_srcs = pickle.load(open(train_srcs,'rb'))
test_srcs = pickle.load(open(test_srcs,'rb'))



train_txt = open(train_txt, 'r', encoding = 'utf-8').read().split('\n\n')

train_tar = []
for u in train_txt:
    train_tar.append('[CLS] ' + u.split('\n')[1])

In [15]:
print(len(train_srcs))
print(len(test_srcs))
print(len(train_tar))

print(train_srcs[4])
print(test_srcs[0])
      

12801
2031
12801
['lastname', 'was', 'admitted', 'to', 'the', 'Neurosurgery', 'service', 'after', 'a', 'study', 'revealed', 'a', 'break', 'in', 'her', '[unused4]', 'catheter', 'at', 'the', 'level', 'of', 'her', 'neck', '.']
['Weaned', 'off', 'vent', 'to', 'Continuous positive airway pressure', 'and', 'was', 'extubated', 'in', 'the', 'afternoon', 'on', '9-2', 'by', 'the', 'lung', 'team', '.']


In [16]:
print(train_srcs[11000])

['#', ')', '[unused4]', ':', 'Patient', 'has', 'history', 'of', 'heavy', 'alcohol', 'abuse', ',', 'and', 'reports', 'having', 'up', 'to', '15-20', 'beers', 'per', 'day', '.']


In [17]:
for i,u in enumerate(train_srcs):
    tmp = ['[CLS]']
    for j,v in enumerate(u):
        if v == '[unused4]':
#             tmp.append('[SEP]')
            tmp.append('[MASK]')
#             tmp.append('[SEP]')
        else:
            tmp.append(v)
    train_srcs[i] = tmp
    
cnt = 0
for i,u in enumerate(test_srcs):
    tmp = ['[CLS]']
    for j,v in enumerate(u):
        if v == '[unused4]':
            cnt += 1
#             tmp.append('[SEP]')
            tmp.append('[MASK]')
#             tmp.append('[SEP]')
        else:
            tmp.append(v)
    test_srcs[i] = tmp

print(cnt)
            
for i,u in enumerate(train_srcs):
    print(' '.join(u))
    if i>5:
        break
for i,u in enumerate(test_srcs):
    print(' '.join(u))
    if i>5:
        break

0
[CLS] Stage 2 [MASK] : On sacrum and scrotum .
[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , bicarbonate 20 and white blood cell 11.3 .
[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .
[CLS] Serum screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants .
[CLS] lastname was admitted to the Neurosurgery service after a study revealed a break in her [MASK] catheter at the level of her neck .
[CLS] Her [MASK] had what was likely a resolving [MASK] .
[CLS] He was instructed to take plavix for only three months .
[CLS] Weaned off vent to Continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung team .
[CLS] She was intubated and was resuscitated after 10-22 minutes of [MASK] arrest .
[CLS] He was given succinyl choline and etomidate for intubation .
[CLS] Had [MASK] and was transfused with appropriate effect .
[CLS] The patient was maintained on logro

In [18]:
train_src_new = []
train_tar_new = []

for src, tar in zip(train_srcs, train_tar):
    src = ' '.join(src)
    src = src.split(' ')
    src_new = src
    if src_new[-1] !='.':
        src_new.append('.')
    tar = tar.split(' ')
    tar_new = []
    p = 0
    flag = 0
    
#     print(' '.join(src_new))
#     print(' '.join(tar))
    for i,u in enumerate(src):
        if u == '[MASK]':
            while p<len(tar) and tar[p]!=src[i-1]:
                tar_new.append(tar[p])
                p+=1
            if p<len(tar):
                tar_new.append(tar[p])
                p+=1
            tar_new.append('[SEP]')
            while p<len(tar) and tar[p]!=src[i+1]:
                tar_new.append(tar[p])
                p+=1
            tar_new.append('[SEP]')
    while p<len(tar):
        tar_new.append(tar[p])
        p+=1
#     print(' '.join(src_new))
#     print(' '.join(tar_new))
#     print('')
    train_src_new.append(src_new)
    train_tar_new.append(tar_new)
    


for i,u in enumerate(train_src_new):
    tmp = []
    for j,v in enumerate(u):
        if v == '[MASK]':
            tmp.append('[SEP]')
            tmp.append('[MASK]')
            tmp.append('[SEP]')
        else:
            tmp.append(v)
    train_src_new[i] = tmp
for i,u in enumerate(test_srcs):
    tmp = []
    for j,v in enumerate(u):
        if v == '[MASK]':
            tmp.append('[SEP]')
            tmp.append('[MASK]')
            tmp.append('[SEP]')
        else:
            tmp.append(v)
    test_srcs[i] = tmp
print(len(train_src_new))
print(len(train_tar_new))

        


12801
12801


In [19]:
print(test_srcs[1])

['[CLS]', 'She', 'was', 'intubated', 'and', 'was', 'resuscitated', 'after', '10-22', 'minutes', 'of', '[SEP]', '[MASK]', '[SEP]', 'arrest', '.']


In [20]:
from bert_embedding import BertEmbedding
import mxnet as mx
import numpy as np
bertembedding = BertEmbedding(ctx=mx.gpu(), batch_size=64, max_seq_length=64)

In [21]:
with open('./data_tmp/abbrs-all-cased.pkl','rb') as f:
    abbrs = pickle.load(f)

In [22]:
train_keys = './data_tmp/train_keys.pkl'
test_keys = './data_tmp/test_keys.pkl'

train_keys = pickle.load(open(train_keys,'rb'))
test_keys = pickle.load(open(test_keys,'rb'))
print(train_keys[:5])
print(test_keys[:5])


[['ulcers'], [], [], [], ['VPS']]
[[], ['PEA'], [], ['anemia'], []]


In [23]:
train_embs = []
for i,u in enumerate(train_srcs):
    p = 0
    for v in u:
        if v== '[MASK]':
            p+=1
    if p!=len(train_keys[i]):
        print(u, train_keys[i])
    if i>30:
        break
    
test_embs = []
for i,u in enumerate(test_srcs):
    p = 0
    for v in u:
        if v== '[MASK]':
            p+=1
    if p!=len(test_keys[i]):
        print(u, test_keys[i])

In [24]:
train_input = []
for u in train_src_new:
    train_input.append(' '.join(u))

test_input = []
for u in test_srcs:
    test_input.append(' '.join(u))

train_output = []
for u in train_tar_new:
    train_output.append(' '.join(u))

print(train_input[:5])
print(train_output[:5])
print(test_input[:5])


['[CLS] Stage 2 [SEP] [MASK] [SEP] : On sacrum and scrotum .', '[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , bicarbonate 20 and white blood cell 11.3 .', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', '[CLS] Serum screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', '[CLS] lastname was admitted to the Neurosurgery service after a study revealed a break in her [SEP] [MASK] [SEP] catheter at the level of her neck .']
['[CLS] Stage 2 [SEP] break in skin [SEP] : On sacrum and scrotum .', '[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , Bicarbonate 20 and white blood cells 11.3 .', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', '[CLS] Serum toxicology test were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', '[CLS] Patient was admitted to the Neurosurgery servic

In [25]:
from transformers import BertTokenizer
from importlib import import_module
import random
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
model_name = 'bert'
x = import_module('models.' + model_name)
config = x.Config(16)
tokenizer = config.tokenizer

def tokenize(input_data):
    ids = []
    labels = []
    masks = []
    for i in tqdm(range(len(input_data))):
        words = tokenizer.tokenize(input_data[i])
        ids.append(words)
    
    ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in ids],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
        
    return ids
 
train_src = tokenize(train_input)
test_src = tokenize(test_input)
train_tar = tokenize(train_output)
train_src_masks = [[float(i != 0.0) for i in ii] for ii in train_src]
test_src_masks = [[float(i != 0.0) for i in ii] for ii in test_src]
train_tar_masks = [[float(i != 0.0) for i in ii] for ii in train_tar]

print(len(train_src))
print(len(test_src))
print(len(train_tar))



pickle.dump(train_src, open("./data_tmp/train_src_23.pkl",'wb'))
pickle.dump(train_src_masks, open("./data_tmp/train_src_masks_23.pkl",'wb'))


pickle.dump(test_src, open("./data_tmp/test_src_23.pkl",'wb'))
pickle.dump(test_src_masks, open("./data_tmp/test_src_masks_23.pkl",'wb'))

pickle.dump(train_tar, open("./data_tmp/train_tar_23.pkl",'wb'))
pickle.dump(train_tar_masks, open("./data_tmp/train_tar_masks_23.pkl",'wb'))






Using TensorFlow backend.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12801/12801 [00:03<00:00, 3422.87it/s]


12801
2031
12801


In [16]:
from tqdm import tqdm
train_embs = []
for (srcs, keys) in tqdm(zip(train_srcs,train_keys)):
    key_embs = []
    for key in keys:
        emb_values = []
        for i, value in enumerate(abbrs[key]):
            if i>=14:
                break
            embs = bertembedding.embedding([value])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
        while len(emb_values)<14:
            embs = bertembedding.embedding(['[PAD] [PAD]'])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
            
        key_embs.append(emb_values)
    train_embs.append(key_embs)
#     if len(train_embs)>=250:
#         break
            

12801it [1:22:56,  2.57it/s]


In [17]:
print(len(train_embs))

12801


In [18]:
pickle.dump(train_embs, open('./data_tmp/train_embs.pkl','wb'))

In [26]:
from tqdm import tqdm
test_embs = []
for (srcs, keys) in tqdm(zip(test_srcs,test_keys)):
    key_embs = []
    for key in keys:
        emb_values = []
        for i, value in enumerate(abbrs[key]):
            if i>=14:
                break
            embs = bertembedding.embedding([value])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
        while len(emb_values)<14:
            embs = bertembedding.embedding(['[PAD] [PAD]'])
            vec = np.mean(embs[0][1], axis=0)
            emb_values.append(vec)
            
        key_embs.append(emb_values)
    test_embs.append(key_embs)
#     if len(train_embs)>=250:
#         break
            

2031it [11:14,  3.01it/s]


In [28]:
pickle.dump(test_embs, open('./data_tmp/test_embs.pkl','wb'))


In [30]:
scores_1 = './data_tmp/test_preds123.pkl'
scores_2 = './data_tmp/test_preds.pkl'

scores_1 = pickle.load(open(scores_1,'rb'))
scores_2 = pickle.load(open(scores_2,'rb'))

In [31]:
def softmax(x):
    if len(x.shape) > 1:
        # 矩阵
        tmp = np.max(x,axis=1) # 得到每行的最大值，用于缩放每行的元素，避免溢出
        x -= tmp.reshape((x.shape[0],1)) # 利用性质缩放元素
        x = np.exp(x) # 计算所有值的指数
        tmp = np.sum(x, axis = 1) # 每行求和        
        x /= tmp.reshape((x.shape[0], 1)) # 求softmax
    else:
        # 向量
        tmp = np.max(x) # 得到最大值
        x -= tmp # 利用最大值缩放数据
        x = np.exp(x) # 对所有元素求指数        
        tmp = np.sum(x) # 求元素和
        x /= tmp # 求somftmax
    return x

for i,u in enumerate(scores_1):
    print(u.asnumpy())
    if i>5:
        break
print(len(scores_1))

[1.9477182e-03 1.2834789e-04 8.4618572e-05 3.7187710e-04 9.9746525e-01
 2.0347509e-06 1.1513179e-09 1.8726982e-09 3.0972069e-09 7.6202554e-08
 4.1563894e-08 1.3514008e-09 1.3164096e-09 3.0789302e-09]
[3.2233633e-03 2.5538218e-04 1.3327619e-05 9.7694927e-01 1.3771916e-07
 1.9558027e-02 3.9938206e-07 2.2134707e-08 1.3474428e-08 8.2160092e-09
 1.1622502e-09 1.0599923e-09 1.7210798e-08 5.0620068e-09]
[9.9295852e-05 9.9963236e-01 2.6836587e-04 3.1079156e-10 2.7242822e-12
 2.9964149e-11 8.2205531e-13 2.7669403e-12 2.4095935e-11 1.0100639e-12
 5.7707268e-12 1.9868390e-11 2.2406469e-12 7.3201384e-12]
[1.1550642e-03 9.9884486e-01 6.9386141e-10 2.0959284e-10 7.6499571e-12
 1.8509590e-11 2.2780376e-12 1.9910597e-12 3.7005638e-11 1.7116905e-11
 3.0358296e-12 3.8686697e-12 2.3394954e-12 1.6520859e-12]
[1.7860773e-05 9.9910921e-01 4.4506535e-04 4.2785192e-04 1.9317308e-11
 4.0426148e-12 1.4503823e-12 2.7345927e-12 7.3280840e-11 6.9505740e-13
 1.8358497e-11 7.1967540e-12 2.8996945e-11 3.0508914e-12]


In [33]:
for i,u in enumerate(scores_2):
    print(u.asnumpy())
    if i>5:
        break
print(len(scores_2))

[8.7378085e-01 1.2614059e-01 6.0028018e-05 1.8474586e-05 3.4883456e-08
 2.5952373e-08 1.6697186e-08 8.8717167e-09 8.1225169e-09 3.4625642e-09
 1.9747584e-09 4.2847116e-08 1.1821275e-09 4.4037147e-08]
[5.1649287e-03 9.6286654e-01 3.4913076e-03 2.8167555e-02 3.0967293e-04
 4.8246065e-09 2.1307014e-09 4.6686472e-09 4.8952207e-09 1.4119332e-09
 7.0544670e-10 5.1246385e-10 5.6442007e-11 2.0266472e-10]
[9.90064722e-03 9.90097761e-01 1.67028384e-06 4.05945499e-09
 3.19267857e-09 1.64763939e-10 7.58567764e-10 9.49660883e-09
 1.07584094e-10 8.38450837e-11 2.38044667e-10 1.35902765e-11
 7.27654673e-11 1.07435623e-10]
[1.0568612e-03 5.5159533e-01 4.4267687e-01 1.5153284e-03 2.8644700e-03
 2.1330939e-04 7.7836528e-05 6.3519917e-10 7.9424254e-09 5.1189799e-11
 1.4415227e-10 1.8019497e-10 1.2605858e-11 4.5535752e-11]
[9.7215682e-02 8.7176198e-01 1.0939831e-02 2.0061418e-02 1.2920410e-06
 1.6896263e-05 1.1870497e-08 2.2082249e-08 8.6618158e-08 2.2316017e-06
 3.8285506e-07 1.1652103e-07 5.4344735e-08 